In [2]:
from google.colab import drive
# drive.mount('/content/drive')

In [50]:
import gzip
import io
import pandas as pd
import re
from datetime import timedelta
import numpy as np

In [190]:
path = '/content/train_test_combined_2012.csv'
pathte = '/content/BPI_Challenge_2012-test.csv'
pathtr = '/content/BPI_Challenge_2012-training.csv'

In [297]:
d = pd.read_csv (path)
dtr = pd.read_csv (pathtr)
dte = pd.read_csv (pathte)

In [298]:
d = d.drop('Unnamed: 0', axis = 1)

In [299]:
d['event time:timestamp']= pd.to_datetime(d['event time:timestamp'])

d = d.sort_values(by=['case concept:name', 'event time:timestamp'])

d = d.assign(nexttime=(d.groupby('case concept:name')['event time:timestamp'].shift(0)
 - d.groupby('case concept:name')['event time:timestamp'].shift(1)).shift(-1)).fillna({'nexttime': 
 np.nan})

d = d.groupby('case concept:name',as_index = False).apply(lambda x: 
  x.reset_index(drop = True)).reset_index().drop('level_0',axis = 1).rename(columns={"level_1": "actionorder"})
d.actionorder = d.actionorder + 1

predictions = d.groupby('actionorder').nexttime.mean()

d = d.assign(prednext=(predictions[d.actionorder].reset_index().nexttime))

In [317]:
d['diff1'] = d.nexttime - d.prednext
d['diff2'] = d.diff1.dt.microseconds + d.diff1.dt.seconds * 10**6 + 864 * 10 ** 8 * d.diff1.dt.days
d['diff3'] = d.diff2 ** 2

'MSE seconds', d.diff3.mean() / (10 ** 6), 'mean seconds', d.diff2.mean() / (10 ** 6)

('MSE seconds', 3.9841262108619116e+18, 'mean seconds', -5.384745075527974e-07)

In [300]:
dtr['event time:timestamp']= pd.to_datetime(dtr['event time:timestamp'])

dtr = dtr.sort_values(by=['case concept:name', 'event time:timestamp'])

dtr = dtr.assign(nexttime=(dtr.groupby('case concept:name')['event time:timestamp'].shift(0)
 - dtr.groupby('case concept:name')['event time:timestamp'].shift(1)).shift(-1)).fillna({'nexttime': 
 np.nan})

dtr = dtr.groupby('case concept:name',as_index = False).apply(lambda x: 
  x.reset_index(drop = True)).reset_index().drop('level_0',axis = 1).rename(columns={"level_1": "actionorder"})
dtr.actionorder = dtr.actionorder + 1

predictions = dtr.groupby('actionorder').nexttime.mean()

# dtr = dtr.assign(prednext=(predictions[dtr.actionorder].reset_index().nexttime))

In [301]:
dte['event time:timestamp']= pd.to_datetime(dte['event time:timestamp'])

dte = dte.sort_values(by=['case concept:name', 'event time:timestamp'])

dte = dte.assign(nexttime=(dte.groupby('case concept:name')['event time:timestamp'].shift(0)
 - dte.groupby('case concept:name')['event time:timestamp'].shift(1)).shift(-1)).fillna({'nexttime': 
 np.nan})

dte = dte.groupby('case concept:name',as_index = False).apply(lambda x: 
  x.reset_index(drop = True)).reset_index().drop('level_0',axis = 1).rename(columns={"level_1": "actionorder"})
dte.actionorder = dte.actionorder + 1

# predictions = dte.groupby('actionorder').nexttime.mean()

dte = dte.assign(prednext=(predictions[dte.actionorder].reset_index().nexttime))

dte['predtime'] = dte['event time:timestamp'] +  dte.prednext

In [304]:
dte['diff1'] = dte.nexttime - dte.prednext
dte['diff2'] = dte.diff1.dt.microseconds + dte.diff1.dt.seconds * 10**6 + 864 * 10 ** 8 * dte.diff1.dt.days
dte['diff3'] = dte.diff2 ** 2

In [316]:
'MSE seconds:', dte.diff3.mean() / (10 ** 6), 'mean seconds:', dte.diff2.mean() / (10 ** 6)

('MSE seconds:', 3.73286735627461e+18, 'mean seconds:', -8528.50257298487)

In [309]:
dexp = dte.drop(['actionorder', 'nexttime', 'prednext', 'diff1', 'diff2', 'diff3'], axis = 1)

In [310]:
dexp

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp,predtime
0,44964012621824,206324,2012-02-03T17:17:11.047+01:00,2500,A_SUBMITTED,COMPLETE,2012-03-02 17:17:11.047,2012-03-02 17:40:42.193799980
1,44964012621825,206324,2012-02-03T17:17:11.047+01:00,2500,A_PARTLYSUBMITTED,COMPLETE,2012-03-02 17:17:11.323,2012-03-07 10:12:32.462567962
2,44964012621826,206324,2012-02-03T17:17:11.047+01:00,2500,A_DECLINED,COMPLETE,2012-03-02 17:17:42.964,2012-03-05 08:20:50.191977503
3,44968307589120,206327,2012-02-03T17:23:41.949+01:00,6000,A_SUBMITTED,COMPLETE,2012-03-02 17:23:41.949,2012-03-02 17:47:13.095799980
4,44968307589121,206327,2012-02-03T17:23:41.949+01:00,6000,A_PARTLYSUBMITTED,COMPLETE,2012-03-02 17:23:42.504,2012-03-07 10:19:03.643567962
...,...,...,...,...,...,...,...,...
47818,56203942035460,214376,2012-02-29T23:51:16.799+01:00,15000,A_DECLINED,COMPLETE,2012-01-03 09:27:37.118,2012-01-08 02:22:58.257567962
47819,56203942035461,214376,2012-02-29T23:51:16.799+01:00,15000,W_Afhandelen leads,COMPLETE,2012-01-03 09:27:41.325,2012-01-06 00:30:48.552977503
47820,56203942035456,214376,2012-02-29T23:51:16.799+01:00,15000,A_SUBMITTED,COMPLETE,2012-02-29 23:51:16.799,2012-03-09 17:09:46.058373441
47821,56203942035457,214376,2012-02-29T23:51:16.799+01:00,15000,A_PARTLYSUBMITTED,COMPLETE,2012-02-29 23:51:17.423,2012-03-01 13:58:31.850780820


In [322]:
dexp.to_csv('test_predicted_time.csv')